In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!git clone https://github.com/ultralytics/yolov5  # Descargar YOLOv5
%cd yolov5
!pip install -r requirements.txt  # Instalar las dependencias

Cloning into 'yolov5'...
remote: Enumerating objects: 17022, done.
remote: Total 17022 (delta 0), reused 0 (delta 0), pack-reused 17022 (from 1)
Receiving objects: 100% (17022/17022), 15.61 MiB | 19.33 MiB/s, done.
Resolving deltas: 100% (11690/11690), done.
/content/yolov5/yolov5


In [12]:
import os
import json
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import cv2
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# Definir rutas en Google Drive (asegúrate de que las carpetas contengan imágenes)
train_dir = '/content/drive/MyDrive/Strawberry Disease Detection Dataset-Usman afzaal/train/images'
test_dir = '/content/drive/MyDrive/Strawberry Disease Detection Dataset-Usman afzaal/test/images'
val_dir = '/content/drive/MyDrive/Strawberry Disease Detection Dataset-Usman afzaal/val/images'
severity_dir = '/content/drive/MyDrive/Strawberry Disease Detection Dataset-Usman afzaal/Test Disease Severity Level'

# Crear directorios de salida si no existen
output_train = '/content/drive/MyDrive/YOLOv5_Annotations/train'
output_test = '/content/drive/MyDrive/YOLOv5_Annotations/test'
output_val = '/content/drive/MyDrive/YOLOv5_Annotations/val'

os.makedirs(output_train, exist_ok=True)
os.makedirs(output_test, exist_ok=True)
os.makedirs(output_val, exist_ok=True)

In [13]:
# Celda n2: Funciones para lectura y generación de máscaras
def read_polygon_coordinates_and_label_from_json(json_path):
    with open(json_path) as f:
        data = json.load(f)
    polygons = data["shapes"]
    coordinates = []
    labels = []
    for polygon in polygons:
        points = polygon["points"]
        label = polygon["label"]
        coordinates.append(points)
        labels.append(label)
    return coordinates, labels

def generate_masks(image_size, coordinates):
    combined_mask = np.zeros(image_size, dtype=np.uint8)
    for points in coordinates:
        points = [(int(x), int(y)) for (x, y) in points]
        if not all(isinstance(point, tuple) and len(point) == 2 for point in points):
            print("Coordenadas no válidas:", points)
            continue
        mask = Image.new("L", image_size, 0)
        ImageDraw.Draw(mask).polygon(points, outline=1, fill=1)
        combined_mask = np.maximum(combined_mask, np.array(mask))
    return combined_mask

def check_image_mask_paths(image_paths, mask_paths):
    print(f"Cantidad de imágenes: {len(image_paths)}")
    print(f"Cantidad de máscaras: {len(mask_paths)}")
    for img_path in image_paths:
        print(f"Imagen: {img_path}")
        mask_path = img_path.replace('images', 'masks').replace('.jpg', '.png')
        print(f"Máscara esperada: {mask_path}")
        if not os.path.exists(mask_path):
            print(f"Máscara no encontrada: {mask_path}")
    assert len(image_paths) == len(mask_paths), "La cantidad de imágenes y máscaras no coincide."

def get_image_paths_masks_and_labels(directory):
    image_paths = []
    masks = []
    labels = []
    for image_path in glob.glob(os.path.join(directory, "*.jpg")):
        json_path = image_path.replace(".jpg", ".json")
        if os.path.exists(json_path):
            coordinates, label = read_polygon_coordinates_and_label_from_json(json_path)
            image = Image.open(image_path)
            image_size = image.size
            image_paths.append(image_path)
            masks.append(generate_masks(image_size, coordinates))
            labels.append(label)
    return image_paths, masks, labels

def get_image_paths_masks_labels_and_severity(directory):
    image_paths = []
    masks = []
    labels = []
    severities = []
    for severity_level in ["Level 1", "Level 2"]:
        level_dir = os.path.join(directory, severity_level)
        for image_path in glob.glob(os.path.join(level_dir, "*.jpg")):
            json_path = image_path.replace(".jpg", ".json")
            if os.path.exists(json_path):
                coordinates, label = read_polygon_coordinates_and_label_from_json(json_path)
                image = Image.open(image_path)
                image_size = image.size
                image_paths.append(image_path)
                masks.append(generate_masks(image_size, coordinates))
                labels.append(label)
                severities.append(severity_level)
    return image_paths, masks, labels, severities

In [14]:
# Celda n3: Dataset de frutillas con transformaciones
import os
import json
from PIL import Image, ImageDraw
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

data_transforms = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

mask_transforms = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor()
])

def draw_mask_on_image(image, mask_json):
    draw = ImageDraw.Draw(image)
    for region in mask_json['shapes']:
        points = region['points']
        polygon_coords = [(int(x), int(y)) for x, y in points]
        draw.polygon(polygon_coords, outline=None, fill=255)
    return image

class StrawberryDataset(Dataset):
    def __init__(self, image_dir, mask_dir, image_transforms=None, mask_transforms=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_transforms = image_transforms
        self.mask_transforms = mask_transforms

        # Crear un diccionario con los nombres de los archivos de imagen sin extensión
        image_files = {f.split(".")[0]: f for f in os.listdir(image_dir) if f.endswith('.jpg')}
        mask_files = {f.split(".")[0]: f for f in os.listdir(mask_dir) if f.endswith('.json')}

        # Solo quedarnos con los archivos que tienen tanto imagen como máscara
        common_files = image_files.keys() & mask_files.keys()

        # Depuración: Imprimir información sobre los archivos
        print(f"Total de archivos de imagen: {len(image_files)}")
        print(f"Total de archivos de máscara: {len(mask_files)}")
        print(f"Archivos comunes: {len(common_files)}")

        # Crear listas de paths para imágenes y máscaras correspondientes
        self.image_paths = [os.path.join(image_dir, image_files[f]) for f in common_files]
        self.mask_paths = [os.path.join(mask_dir, mask_files[f]) for f in common_files]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Cargar la imagen y la máscara
        image_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]

        # Cargar la imagen y aplicar transformaciones
        image = Image.open(image_path).convert("RGB")
        if self.image_transforms:
            image = self.image_transforms(image)

        # Cargar la máscara desde el archivo JSON
        with open(mask_path, 'r') as f:
            mask_data = json.load(f)

        # Crear una imagen en blanco para la máscara (del mismo tamaño que la imagen)
        mask = Image.new('L', (image.width, image.height))
        draw = ImageDraw.Draw(mask)

        # Dibujar la máscara basada en los puntos del archivo JSON
        for shape in mask_data['shapes']:
            points = shape['points']
            draw.polygon(points, outline=1, fill=1)

        if self.mask_transforms:
            mask = self.mask_transforms(mask)

        return image, mask

# Nueva función: preparar_targets para YOLOv5
def preparar_targets(masks, image_size):
    targets = []
    for mask in masks:
        mask = mask.numpy()  # Convertir el tensor de PyTorch a numpy
        object_indices = np.where(mask == 1)
        if len(object_indices[0]) > 0:
            x_min, x_max = np.min(object_indices[1]), np.max(object_indices[1])
            y_min, y_max = np.min(object_indices[0]), np.max(object_indices[0])
            # Normalizar las coordenadas
            bbox = [x_min / image_size[0], y_min / image_size[1], x_max / image_size[0], y_max / image_size[1]]
            targets.append(bbox)
        else:
            targets.append([0, 0, 0, 0])  # Si no hay objeto, añades un bbox vacío
    return torch.tensor(targets, dtype=torch.float32)


In [15]:
import os

# Ruta de la carpeta de las máscaras
mask_dir = '/content/drive/MyDrive/Strawberry Disease Detection Dataset-Usman afzaal/train/masks'

# Listar los archivos en la carpeta de máscaras
mask_files = [f for f in os.listdir(mask_dir) if f.endswith('.json')]

# Depuración: Mostrar los primeros 10 archivos en la carpeta de máscaras
print(f"Total de archivos de máscara: {len(mask_files)}")
print("Algunos archivos de máscara:", mask_files[:10])

# Verifica si hay archivos con extensión ".JSON" en mayúsculas
mask_files_upper = [f for f in os.listdir(mask_dir) if f.endswith('.JSON')]
print(f"Total de archivos de máscara con .JSON en mayúsculas: {len(mask_files_upper)}")

Total de archivos de máscara: 1450
Algunos archivos de máscara: ['blossom_blight89.json', 'gray_mold235.json', 'gray_mold120.json', 'gray_mold129.json', 'gray_mold217.json', 'blossom_blight83.json', 'gray_mold221.json', 'gray_mold139.json', 'gray_mold23.json', 'blossom_blight99.json']
Total de archivos de máscara con .JSON en mayúsculas: 0


In [16]:
# Celda n4: Modelo de entrenamiento
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import joblib
import os

from models.yolo import Model as YOLOv5

total_epochs = 60
batch_size = 10

model = YOLOv5(cfg='/content/yolov5/models/yolov5s.yaml')

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

def train_model(model, train_loader, val_loader, total_epochs, device):
    model.to(device)
    for epoch in range(total_epochs):
        model.train()
        running_loss = 0.0
        for i, (images, masks) in enumerate(train_loader):
            images = images.to(device)
            masks = masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            targets = preparar_targets(masks, images.shape[2:])
            targets = targets.to(device)
            total_loss = criterion(outputs, targets.float())
            total_loss.backward()
            optimizer.step()
            running_loss += total_loss.item()
        print(f"Época {epoch+1}/{total_epochs}, Pérdida: {running_loss / len(train_loader)}")
        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for images, masks in val_loader:
                images = images.to(device)
                masks = masks.to(device)
                outputs = model(images)
                targets = preparar_targets(masks, images.shape[2:])
                targets = targets.to(device)
                loss = criterion(outputs, targets.float())
                val_loss += loss.item()
        print(f"Pérdida de validación: {val_loss / len(val_loader)}")


                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  1   1182720  

In [17]:
# Definir las rutas para los directorios de imágenes y máscaras
train_image_dir = '/content/drive/MyDrive/Strawberry Disease Detection Dataset-Usman afzaal/train/images'
train_mask_dir = '/content/drive/MyDrive/Strawberry Disease Detection Dataset-Usman afzaal/train/masks'

val_image_dir = '/content/drive/MyDrive/Strawberry Disease Detection Dataset-Usman afzaal/val/images'
val_mask_dir = '/content/drive/MyDrive/Strawberry Disease Detection Dataset-Usman afzaal/val/masks'

# Crear los datasets de entrenamiento y validación
train_dataset = StrawberryDataset(train_image_dir, train_mask_dir, image_transforms=data_transforms, mask_transforms=mask_transforms)
val_dataset = StrawberryDataset(val_image_dir, val_mask_dir, image_transforms=data_transforms, mask_transforms=mask_transforms)

# Crear los DataLoader para cargar los datos en lotes
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

Total de archivos de imagen: 1450
Total de archivos de máscara: 1450
Archivos comunes: 1450
Total de archivos de imagen: 307
Total de archivos de máscara: 307
Archivos comunes: 307


In [18]:
import os
import joblib

# Definir la ruta donde guardar el modelo
save_dir = "/content/drive/MyDrive/YOLOv5_Models"

# Verificar si el directorio existe, si no, crearlo
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Guardar el modelo
model_path = os.path.join(save_dir, "strawberry_model_v2.pth")
joblib.dump(model.state_dict(), model_path)

print(f"Modelo guardado en: {model_path}")

Modelo guardado en: /content/drive/MyDrive/YOLOv5_Models/strawberry_model_v2.pth
